In [4]:
import PIL

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = './train/'
validation_data_dir = './valid/'
nb_train_samples = 2000
nb_validation_samples = 800
nb_epoch = 50


model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, img_width, img_height)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(8))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

model.load_weights('first_try.h5')

Found 3777 images belonging to 8 classes.
Found 80 images belonging to 8 classes.
Epoch 1/50
1984/2000 [============================>.] - ETA: 1s - loss: 1.7577 - acc: 0.4395

/Users/Masters/anaconda2/lib/python2.7/site-packages/keras/engine/training.py:1527: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


2016/2000 [==============================] - 195s - loss: 1.7589 - acc: 0.4400 - val_loss: 2.1481 - val_acc: 0.2000
Epoch 2/50
2017/2000 [==============================] - 182s - loss: 1.5991 - acc: 0.4398 - val_loss: 2.0826 - val_acc: 0.2475
Epoch 3/50
2016/2000 [==============================] - 435s - loss: 1.5101 - acc: 0.5089 - val_loss: 1.8872 - val_acc: 0.2650
Epoch 4/50
2017/2000 [==============================] - 178s - loss: 1.4279 - acc: 0.5335 - val_loss: 1.7690 - val_acc: 0.2375
Epoch 5/50
2016/2000 [==============================] - 177s - loss: 1.3417 - acc: 0.5387 - val_loss: 1.7084 - val_acc: 0.3287
Epoch 6/50
2017/2000 [==============================] - 179s - loss: 1.2989 - acc: 0.5568 - val_loss: 1.5505 - val_acc: 0.3337
Epoch 7/50
2016/2000 [==============================] - 181s - loss: 1.1894 - acc: 0.5908 - val_loss: 1.5445 - val_acc: 0.3375
Epoch 8/50
2017/2000 [==============================] - 183s - loss: 1.1781 - acc: 0.6009 - val_loss: 1.6389 - val_acc: 0.

IOError: Unable to open file (Unable to open file: name = 'first_try.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [7]:
model.save_weights('first_try.h5')  # always save your weights after training or during training

In [8]:
model.save('my_model.h5') 

In [57]:
import glob
import os
import cv2
import numpy as np

def load_test_imgs(dir):
    images = []
    labels = []
    list_imgs = glob.glob(dir+"/*.jpg")
    for subdir in list_imgs:
        img = cv2.imread(subdir)  
        resized = cv2.resize(img, (150, 150))
        x = img_to_array(resized)  # this is a Numpy array with shape (3, 150, 150)
        #x = x.reshape((1,) + x.shape)
        x = x/255.0
        flbase = os.path.basename(subdir)
        labels.append(flbase)
        images.append(x)
    return images, labels

        

In [61]:
test_images, test_labels = load_test_imgs("./test_stg1/")

In [62]:
test_images = np.array(test_images)

In [63]:
test_images.shape

(1000, 3, 150, 150)

In [64]:
predictions = model.predict_proba(test_images)

1000/1000 [==============================] - 8s     


In [65]:
predictions.shape

(1000, 8)

In [68]:
import pandas as pd

df = pd.DataFrame({'image': test_labels, 
                   'ALB': predictions[:,0],	
                   'BET': predictions[:,1],
                   'DOL': predictions[:,2],
                   'LAG': predictions[:,3],
                   'NoF': predictions[:,4],
                   'OTHER': predictions[:,5],
                   'SHARK': predictions[:,6],
                   'YFT': predictions[:,7]})

In [70]:
df.to_csv("./first_submission.csv")